## LOAD FULL PAGE WITH SELENIUM

In [1]:
%run ./app1-crawler.py

load more button clicked!
load more button clicked!
load more button clicked!
load more button clicked!
load more button clicked!
html file saved!


## GET ALL ARTICLES AND LINKS

In [2]:
# Read HTML

from bs4 import BeautifulSoup
import codecs

file = codecs.open("./my_page.html", "r", "utf-8")
soup = BeautifulSoup(file, 'html.parser')
file.close()

In [3]:
# Remove ads

try:
    for element in soup.find('div', {'class':'articles'}).findAll('a',{'class':'article-block wnad-artbnr'}):
        element.decompose()
except:
    pass

try:
    for element in soup.find('div', {'class':'articles'}).findAll('a',{'class':'article-block wnad-article'}):
        element.decompose()
except:
    pass

In [4]:
articles = soup.find('div',{'class':'articles'}).findAll('a',{'class':'article-block'})
print(len(articles))

120


In [ ]:
import re

titles = []
links = []
article_years = []
article_months = []
years = ['2021', '2022']
months = ['一月','二月','三月','四月','五月','六月','七月','八月','九月','十月','nov','dec']

year_months = []
for year in years:
  for month in months:
    year_months.append([year, month])


for article in articles:

# GET TITLE & LINKS
  try:
    title = article.find('h2').text
    title = re.sub('【蘇珊米勒】','', title)
  except:
    title = 'NO TITLE'
  #print(title)

  if any([x in title for x in years]):
    titles.append(title)
    link = "https://womany.net" + article.get('href')
    links.append(link)

  titles2 = titles

for title in titles2:
  try:
    title = re.sub('十二月','dec', title)
  except:
    pass
  
  try:
    title = re.sub('十一月','nov', title)
  except:
    pass

  print(title)

  for ym in year_months: 
    if all([x in title for x in ym]):
      article_years.append(ym[0])
      article_months.append(str(months.index(ym[1])+1))



In [6]:
print(len(article_years))
print(len(titles2))

96
96


In [7]:
import pandas as pd

data = zip(article_years, article_months, titles, links)
df = pd.DataFrame(list(data), columns=['article_year','article_month','title','link'])

In [8]:
df.head(5)

,article_year,article_month,title,link
0,2022,12,2022 雙魚、巨蟹、天蠍：水象星座十二月運勢,https://womany.net/read/article/31336?type=int...
1,2022,12,2022 雙子、天秤、水瓶：風象星座十二月運勢,https://womany.net/read/article/31335?type=int...
2,2022,12,2022 牡羊、獅子、射手：火象星座十二月運勢,https://womany.net/read/article/31333?type=int...
3,2022,12,2022 金牛、處女、摩羯：土象星座十二月運勢,https://womany.net/read/article/31332?type=int...
4,2022,11,2022 雙子、天秤、水瓶：風象星座十一月運勢,https://womany.net/read/article/31200?type=int...


In [9]:
df.shape

(96, 4)

In [10]:
df.to_csv("articles.csv", sep="|", index=False)

In [11]:
df2 = pd.read_csv('articles.csv', sep='|')
df2.head(6)

,article_year,article_month,title,link
0,2022,12,2022 雙魚、巨蟹、天蠍：水象星座十二月運勢,https://womany.net/read/article/31336?type=int...
1,2022,12,2022 雙子、天秤、水瓶：風象星座十二月運勢,https://womany.net/read/article/31335?type=int...
2,2022,12,2022 牡羊、獅子、射手：火象星座十二月運勢,https://womany.net/read/article/31333?type=int...
3,2022,12,2022 金牛、處女、摩羯：土象星座十二月運勢,https://womany.net/read/article/31332?type=int...
4,2022,11,2022 雙子、天秤、水瓶：風象星座十一月運勢,https://womany.net/read/article/31200?type=int...
5,2022,11,2022 金牛、處女、摩羯：土象星座十一月運勢,https://womany.net/read/article/31199?type=int...


## GET CONTENT FOR EACH HOROSCOPE IN EACH ARTICLE LINK

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

df = pd.read_csv('articles.csv', sep='|')
df.head(6)

,article_year,article_month,title,link
0,2022,12,2022 雙魚、巨蟹、天蠍：水象星座十二月運勢,https://womany.net/read/article/31336?type=int...
1,2022,12,2022 雙子、天秤、水瓶：風象星座十二月運勢,https://womany.net/read/article/31335?type=int...
2,2022,12,2022 牡羊、獅子、射手：火象星座十二月運勢,https://womany.net/read/article/31333?type=int...
3,2022,12,2022 金牛、處女、摩羯：土象星座十二月運勢,https://womany.net/read/article/31332?type=int...
4,2022,11,2022 雙子、天秤、水瓶：風象星座十一月運勢,https://womany.net/read/article/31200?type=int...
5,2022,11,2022 金牛、處女、摩羯：土象星座十一月運勢,https://womany.net/read/article/31199?type=int...


In [14]:
df = df.reset_index()

In [15]:
def clean_page(page):

  try:
    for img in page.findAll('p', {'class':'with_img'}):
      img.decompose()
  except:
    pass

  try:
    for ad in page.findAll('p', {'class':'ads'}):
      ad.decompose()
  except:
    pass

  try:
    for p in page.findAll('span', {'class':'further'}):
      p.decompose()
  except:
    pass
  return page

In [16]:
def extract_content(page, extras):  

  # get titles and title tags
  title_tags = []

  for tag in page.findAll('h2'):
    if tag.has_attr('id'):
      title_tags.append(tag)

  for tag in page.findAll('h3'):
    if tag.has_attr('id'):
      title_tags.append(tag)

  titles = [re.sub("（.*）", "", tag.text) for tag in title_tags]

  texts = []

  # extract content one by one
  for title_tag in title_tags:

    text = ""
    for tag in title_tag.next_siblings:

      if (tag.next_sibling is None) or (tag.next_sibling in title_tags):
        break;
      elif tag.name == 'p':
        text += tag.text
  
    for extra in extras:
      text = re.sub(extra, '', text)

    texts.append(text)

  return list(zip(titles, texts))

In [17]:
article_year = []
article_month = []
horoscope = []
content = []
extras = ['翻譯：.*',' ',r'\n',r'\xa0']

for url in df['link']:
  r = requests.get(url)
  soup = BeautifulSoup(r.text,'lxml')

  page = soup.find('section', {'class':'article-body'})

  clean_page(page)

  extract_content(page, extras)

  for h, c in extract_content(page,extras):
    article_year.append(df.loc[df['link'] == url, 'article_year'].iloc[0])
    article_month.append(df.loc[df['link'] == url, 'article_month'].iloc[0])
    horoscope.append(h)
    content.append(c)
  
  time.sleep(2)

data = zip(article_year, article_month, horoscope, content)
df2 = pd.DataFrame(list(data), columns=['article_year','article_month','horoscope','content'])

In [18]:
df2.shape

(288, 4)

In [19]:
df2.head(10)

,article_year,article_month,horoscope,content
0,2022,12,雙魚座,家庭仍舊是你的生活重心，在2023年3月25日之前，家族中的某人依然需要你多加關注。這主要是...
1,2022,12,巨蟹座,說到巨蟹座的工作啊，你一直選擇默默無聞地在幕後付出，作為掌控全局的角色。巨蟹座領導會毫不吝嗇...
2,2022,12,天蠍座,天蠍座有兩個守護星，這兩顆行星與其他行星的互動通常對你的運勢非常重要。第一個守護星火星正位於...
3,2022,12,雙子座,火星，這顆高能量的行星，已經在你的星座停留了異常長的時間，在8月20日首次進入到雙子座，並一...
4,2022,12,天秤座,你關注的焦點並不局限於本土，而是放眼全球的，因為火星從8月20日起就在你的第九宮國際關係與企...
5,2022,12,水瓶座,一進入12月，就會感受到節日的熱鬧。你最喜歡的會是12月7日的那個雙子座滿月（影響期：12月...
6,2022,12,牡羊座,假期即將到來，而白羊座們正迫切地需要把買買買任務盡快完成，以免因為供應問題導緻禮物沒有到貨。...
7,2022,12,獅子座,眾所周知，你是一個熱心慷慨的人。難怪你的朋友都喜歡你。獅子座在黃道十二宮中的角色是教會其他人...
8,2022,12,射手座,對你來說這是一個非常值得紀念的童話般美好的假日月份。8月20日，掌管著能量同時也是你第五宮宮...
9,2022,12,金牛座,前段時間，在10月27日，木星，禮物和運氣的給予者，逆回到雙魚座，你第十一宮快樂，幸福，希望...


In [20]:
df2.to_csv("horoscopes.csv", sep="|", index=False)

In [21]:
df3 = pd.read_csv('horoscopes.csv', sep='|')
df3.head(6)

,article_year,article_month,horoscope,content
0,2022,12,雙魚座,家庭仍舊是你的生活重心，在2023年3月25日之前，家族中的某人依然需要你多加關注。這主要是...
1,2022,12,巨蟹座,說到巨蟹座的工作啊，你一直選擇默默無聞地在幕後付出，作為掌控全局的角色。巨蟹座領導會毫不吝嗇...
2,2022,12,天蠍座,天蠍座有兩個守護星，這兩顆行星與其他行星的互動通常對你的運勢非常重要。第一個守護星火星正位於...
3,2022,12,雙子座,火星，這顆高能量的行星，已經在你的星座停留了異常長的時間，在8月20日首次進入到雙子座，並一...
4,2022,12,天秤座,你關注的焦點並不局限於本土，而是放眼全球的，因為火星從8月20日起就在你的第九宮國際關係與企...
5,2022,12,水瓶座,一進入12月，就會感受到節日的熱鬧。你最喜歡的會是12月7日的那個雙子座滿月（影響期：12月...
